# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [68]:
import os
import glob
import json
import psycopg2
import numpy as np
import pandas as pd
from sql_queries import songplay_table_insert, user_table_insert, song_table_insert, artist_table_insert, time_table_insert, song_select
from datetime import datetime

In [69]:
conn = psycopg2.connect("user=postgres \
                            host=127.0.0.1 \
                            port=5432 \
                            dbname=sparkify_db \
                            password=passryme1")
cur = conn.cursor()

In [70]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [71]:
# song_files = data/song_data

In [72]:
filepath = "data/song_data"

In [73]:
songs = get_files(filepath)

In [74]:
df = pd.read_json(songs[0], lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [76]:
song_list = []
for song in songs:
    with open(song) as file:
        file_load = json.load(file)
        song_list.append(file_load)

df2 = pd.DataFrame(song_list)
df2.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


In [77]:
conn.set_session(autocommit=True)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [78]:
for value in df2.values:
    num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = value
    
    # insert song record
    song_data = (song_id, title, artist_id, year, duration)
    cur.execute(song_table_insert, song_data)


print(f"Records inserted for file {filepath}")

Records inserted for file data/song_data


Run `test.ipynb` to see if you've successfully added a record to this table.

In [79]:
df2.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [80]:
for value in df2.values:
    num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = value
    
    # insert artist record
    artist_data = (artist_id, artist_name, artist_location, artist_latitude, artist_longitude)
    cur.execute(artist_table_insert, artist_data)
    

print(f"Records inserted for file {filepath}")

Records inserted for file data/song_data


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [81]:
filepath = "data/log_data"

In [82]:
logs = get_files(filepath)

In [83]:
df3 = pd.read_json(logs[0], lines=True)
df3.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [84]:
for log in logs:
    df = pd.read_json(log, lines=True)
    df = df[df['page'] == "NextSong"].astype({'ts': 'datetime64[ms]'})
    
    for value in df.values:
        artist, auth, first_name, gender, item_in_session, last_name, \
        length, level, location, method, page, registration, \
        session_id, song, status, ts, user_agent, user_id = value
    
        # insert artist record
        if user_id == '':
            user_id = "NULL"
            cur.execute(user_table_insert, user_data)
        else:
            user_data = (user_id, first_name, last_name, gender, level)
            cur.execute(user_table_insert, user_data)
    

print(f"Records inserted for file {filepath}")


Records inserted for file data/log_data


Run `test.ipynb` to see if you've successfully added records to this table.

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [85]:
for log in logs:
    df = pd.read_json(log, lines=True) 
    df = df[df['page'] == "NextSong"].astype({'ts': 'datetime64[ms]'})
    
    for value in df.values:
        artist, auth, first_name, gender, item_in_session, last_name, \
        length, level, location, method, page, registration, \
        session_id, song, status, ts, user_agent, user_id = value
        
        time_data = (ts, ts.hour, ts.day, ts.weekofyear, ts.month, ts.year, ts.day_name())
        
        # insert time record
        cur.execute(time_table_insert, time_data)
        

print(f"Records inserted for file {filepath}")

Records inserted for file data/log_data


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [86]:
for log in logs:
    df = pd.read_json(log, lines=True) 
    df = df[df['page'] == "NextSong"].astype({'ts': 'datetime64[ms]'})
    
    for value in df.values:
        artist, auth, first_name, gender, item_in_session, last_name, \
        length, level, location, method, page, registration, \
        session_id, song, status, ts, user_agent, user_id = value
        
        start_time = ts
    
        cur.execute(song_select, (song, artist, length))
        results = cur.fetchone()
        
        if results:
            song_id, artist_id = results
        else:
            song_id, artist_id = None, None

        # insert songplay record
        songplay_data = (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
        cur.execute(songplay_table_insert, songplay_data)
    

print(f"Records inserted for file {filepath}")

Records inserted for file data/log_data


Run `test.ipynb` to see if you've successfully added records to this table.

In [87]:
cur.execute("""DROP TABLE IF EXISTS songplays""")

In [88]:
cur.execute("""DROP TABLE IF EXISTS songs""")
cur.execute("""DROP TABLE IF EXISTS artists""")
cur.execute("""DROP TABLE IF EXISTS users""")
cur.execute("""DROP TABLE IF EXISTS time""")

# Close Connection to Sparkify Database

In [89]:
conn.close()
cur.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.